In [869]:
# Restructure of program into more effective functions
#
#
#

In [870]:
# Imports
import pandas as pd
import numpy as np
import requests
import json
from json import loads
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline
import time

# Charting-specific imports (matplotlib, matplotlib.finance)
import matplotlib.finance
import matplotlib.ticker as ticker
from matplotlib.finance import candlestick_ohlc,candlestick2_ohlc
#from matplotlib.finance import volume_overlay, volume_overlay2
from matplotlib.dates import  DateFormatter, epoch2num
    # https://matplotlib.org/api/finance_api.html#module-matplotlib.finance
import matplotlib.patheffects as PathEffects
    # [plt.text object].set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])
    # plt.text reference: https://matplotlib.org/api/text_api.html#matplotlib.text.Text

# API-specific imports (local install required; do NOT use default pip install)
import gdax
    # Python setup.py install with environment activated to install/use
    # Install locally with 'python setup.py install' & development branch of gdax-python checked out
    # Do not use default gdax pip install package - that version of the package is currently broken
        # Default pip install has broken mongo connection and websocket connection close() error

# Pymongo import for connection to local client DB
import pymongo
from pymongo import MongoClient

# Preprocessing Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from keras.utils import to_categorical 

# ML Imports 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding, Flatten
from keras.layers import LSTM, GRU
from keras.models import load_model
from keras.models import model_from_json
from keras import backend as K

# autoSR() function import requirements
from sklearn.cluster import MeanShift, estimate_bandwidth
from pandas_datareader import data, wb

###########################################################################
### Force Keras/TF to use CPU backend when GPU present by setting:
    # {'CPU' : 1, 'GPU' : 0}
    
#num_cores = 4
#config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        #inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        #device_count = {'CPU' : 1, 'GPU' : 1})
#session = tf.Session(config=config)
#K.set_session(session)
###########################################################################

# Import to check check for GPU availability for tensorflow backend
from tensorflow.python.client import device_lib

# Verify GPU availability for tensorflow backend
print(device_lib.list_local_devices())
print("==============================================")
print(K.tensorflow_backend._get_available_gpus())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1807914024305920931
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9222031934
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9893350297991879003
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
['/job:localhost/replica:0/task:0/device:GPU:0']


In [871]:
# Global variables

# Boolean to drop existing mongo collection/scrape upon scrape() init
dropFlag = False

# Boolean to set size_delta to l2update values for first update to snapshot
firstUpdate_bids = False
firstUpdate_asks = False
firstUpdate_both = False

# Value to track if feature_creation_inital() was run
    # Inital value = False
inital_feature_run = False

In [872]:
# Connection establishment

# Establish connection to GDAX public endpoint
public_client = gdax.PublicClient()

# Mongo database and collection specification:
mongo_client = MongoClient('mongodb://localhost:27017/')
db = mongo_client.btcusd_db
btcusd_collection = db.btcusd_collection

In [873]:
# Function to start scrape process from websocket to mongodb instance
def scrape_start():
    
    # Drop existing collection from db if dropFlag == True (on new scrape):
    if 'btcusd_db' in mongo_client.database_names() and dropFlag is True:
        mongo_client['btcusd_db'].drop_collection('btcusd_collection')
        #print(mongo_client.database_names())
        #print(db.collection_names())
        
    # Start instance of websocket client for L2 Orderbook + L2 update data request and scrape
    wsClient = gdax.WebsocketClient(url="wss://ws-feed.gdax.com", 
                                products=["BTC-USD"],\
                                message_type="subscribe",\
                                channels =["level2"],\
                                mongo_collection=btcusd_collection,\
                                should_print=False)
    
    # Save request open time and start websocket
    time.sleep(4)
    request_time_start=public_client.get_time()
    wsClient.start()
    
    # scrape_time is variable for time between websocket connection start and end
        # Defined in seconds
        # i.e. 600 seconds = scrape running for 10 minutes
    scrape_time = 600

    time.sleep(scrape_time)
    # Save request close time and close websocket
    request_time_end=public_client.get_time()
    wsClient.close()
    
    # Append request times for open/close of websocket stream to dataframe, save to csv
    request_log_df = pd.DataFrame.from_dict({'request start':request_time_start,'request end':request_time_end},orient ='index')
    request_log_df.to_csv("raw_data/request_log.csv",header=True,encoding='utf-8',index =True)

# Function to load and parse data from Mongo into dataframes
def load_parse():
    
    #Collection specification (in database)
    input_data = db.btcusd_collection 
    
    # Create individual dataframes for main response types: snapshot, l2update
    snapshot = pd.DataFrame(list(input_data.find({'type':'snapshot'})))
    l2update = pd.DataFrame(list(input_data.find({'type':'l2update'})))
    
    ### snapshot  response load and parse ###
    
    # Extract asks/bid individual column of array of arrays into lists
    snapshot_asks = snapshot[['asks'][0]][0]
    snapshot_bids = snapshot[['bids'][0]][0]
    
    # Convert list (of array of arrays) into dataframe
    snapshot_asks_df =pd.DataFrame(snapshot_asks)
    snapshot_bids_df =pd.DataFrame(snapshot_bids)
    
    # Rename columns to snapshot array format:
        # snapshot array format: [price, size]
            # [side, price, size] format 
        # Ask = sell price, bid = buy price
    snapshot_asks_df.rename(columns ={0:'price',1:'size'}, inplace =True)
    snapshot_bids_df.rename(columns ={0:'price',1:'size'}, inplace =True)
    snapshot_asks_df['side'] = "sell"
    snapshot_bids_df['side'] = "buy"
    cols =['side','price','size']
    snapshot_asks_df = snapshot_asks_df[cols]
    snapshot_bids_df = snapshot_bids_df[cols]
    
    ### L2 update response load and parse ###
    
    # Restucture l2update to have [side,price,size] from 'changes' column
    l2update_clean = l2update[['changes','time']]
          
    # Convert changes list of lists -> into array 
    l2_array = np.ravel(l2update_clean['changes']) 
    # Flatten the list and remove outer bracket:
    flattened = [val for sublist in l2_array for val in sublist]
        # Reference: https://stackoverflow.com/questions/11264684/flatten-list-of-lists?
    # Convert back to dataframe and combine with timestamps from l2update:
    changes_df= pd.DataFrame.from_records(flattened)
    # Add time column back to L2 update dataframe
    l2update_formatted = pd.concat([changes_df,l2update_clean['time']],1)
    # Rename columns for [side, price, size]:
    l2update_formatted.rename({0:"side",1:"price",2:"size"}, axis ='columns',inplace=True)
    
    # Save parsed data to csv (API -> Mongo -> Dataframe -> .csv)
    #save_csv()
    
    # Save data to .csv format in raw_data folder
    l2update_formatted.to_csv("raw_data/l2update.csv",header=True,encoding='utf-8',index =False)
    snapshot_asks_df.to_csv("raw_data/snapshot_asks.csv",header=True,encoding='utf-8',index =False)
    snapshot_bids_df.to_csv("raw_data/snapshot_bids.csv",header=True,encoding='utf-8',index =False)

In [874]:
# Function to read in raw data from csv, 'raw_data' folder
    # 'raw_data' folder has 1hr of scraped data (snapshot + l2 response updates)
    # 'test_data' folder only has 10 minutes of scraped data 
    
def raw_csv_load():
    global request_log_df
    global snapshot_asks_df
    global snapshot_bids_df
    global l2update_df
    
    global l2update_15min_1
    global l2update_15min_2
    global l2update_15min_3
    global l2update_15min_4
  
    snapshot_asks_df = pd.read_csv("raw_data/snapshot_asks.csv",dtype ={'size':float,'size_delta':float})
    snapshot_bids_df = pd.read_csv("raw_data/snapshot_bids.csv",dtype ={'size':float,'size_delta':float})
    l2update_df = pd.read_csv("raw_data/l2update.csv", dtype ={'size':float})
    request_log_df= pd.read_csv("raw_data/request_log.csv")
    
    #l2update_df (1 hour of data) split into 4 15-min dataframes/csvs
    l2update_15min_1 = pd.read_csv("raw_data/l2update_15min_1.csv",dtype ={'size':float})
    l2update_15min_2 = pd.read_csv("raw_data/l2update_15min_2.csv",dtype ={'size':float})
    l2update_15min_3 = pd.read_csv("raw_data/l2update_15min_3.csv",dtype ={'size':float})
    l2update_15min_4 = pd.read_csv("raw_data/l2update_15min_4.csv",dtype ={'size':float})


In [875]:
# Function that requests 15min of chart data from GDAX API 
    # Start and end time can be specified, but must be in ISO 8601 format
    
def chart_15m_request(request_log_df):
    
    global chart_15m_df
    request_start = request_log_df['iso'][1]
    request_end = request_log_df['iso'][0]

    # Request 15 minutes of candlestick (open high low close) data from API
        # start time must be in ISO 8601 format for get_product_historic_rates()
    chart_15m = public_client.get_product_historic_rates('BTC-USD', start = request_start, granularity=60)

    # Convert chart data response into dataframes
    chart_15m_df =pd.DataFrame.from_records(chart_15m,columns=["time","low", "high","open" ,"close", "volume"])

    # Reorder columns according to label order required by matplotlib finance package
    chart_15m_df = chart_15m_df[["time","open","high","low","close","volume"]]

    # Reverse/sort timestamp order (without reversal chart labels/axis will be out of order)
    chart_15m_df.sort_values(by='time',axis=0, inplace =True)

    #Convert/sort time to datetime object (for matplotlib chart format requirement)
    chart_15m_df['time'] =pd.to_datetime(chart_15m_df['time'],unit='s')

In [876]:
# Auto support/resistance adapted into function 
    # Using estimate_bandwidth and meanshift() from sklearn.cluster library
    # Source: Adapted from https://github.com/nakulnayyar/SupResGenerator
    # Takes dataframe object with open/high/low/close values 
        # Returns list of auto-generated support and resistance levels for price action
        
def autoSR(dataframe):
    
    global ml_results_modified
    data = chart_15m_df.as_matrix(columns=['close'])
    data2 = data[:len(data)*1]
    bandwidth = estimate_bandwidth(data2, quantile=0.1, n_samples=100)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(data2)
    ml_results = []
    for k in range(len(np.unique(ms.labels_))):
            my_members = ms.labels_ == k
            values = data[my_members, 0]    
            ml_results.append(min(values))
            ml_results.append(max(values))
            # Remove duplicate S/R level values
            # Using sets
            ml_set =set(ml_results)
            ml_results = list(ml_set)
            # Sort values before return output 
            ml_results.sort()
    
    # Convert ml_results into sorted int array
    ml_results_modified =np.asarray(ml_results)
    ml_results_modified =np.trunc(ml_results_modified).astype(int)
    ml_results_modified = np.sort(ml_results_modified,kind = 'quicksort') 
    # Remove duplicates from int conversion
    ml_results_modified_set=set(ml_results_modified)
    ml_results_modified = list(ml_results_modified_set)
    ml_results_modified = np.sort(ml_results_modified,kind = 'quicksort')
    ml_results_modified
            
    return ml_results_modified

In [877]:
# Function to generate candlestick chart from candlestick data (API request)
    # Calls autoSR() inside function to also plot support/resistance lines
    
def generate_chart(dataframe):

    matplotlib.rcParams.update({'font.size': 14})
    dataframe = chart_15m_df 
    
    # Generate chart of past 15 minutes with autogenerated support and resistance levels:
    fig, ax = plt.subplots(figsize=(20, 14))
    candlestick2_ohlc(ax,chart_15m_df['open'],chart_15m_df['high'],\
                      chart_15m_df['low'],chart_15m_df['close'],width=2,\
                      colorup='k',colordown='r',alpha=.5)
    
    ax.set_xticklabels(chart_15m_df['time'] ,rotation=30,ha="right")
    
    ax.xaxis.set_major_locator(ticker.MaxNLocator(17))
        # n + 2 for proper label set at 20,18/20,14 chart size
    plt.yticks(np.arange(int(min(chart_15m_df['low'])-10), max(chart_15m_df['high'])+10, 10))
    plt.xlabel('Time')
    plt.ylabel('Price (USD)')
    #ax.minorticks_on()
    ax.set_axisbelow(True)
    #fig.set_zorder(0)
    #ax.set_zorder(1)
    ax.tick_params(axis='y', pad=10)
    ax.grid(which='major', linestyle='--', linewidth='0.2', color='b')
    matplotlib.pyplot.title("15 Min BTC/USD with Support/Resistance Levels")
    ml_results_modified = autoSR(chart_15m_df)
    
    # Add S/R labels to chart
        # count increment is used for staggering of labels in conjunction with modulus
        
    count = 1
    for k in ml_results_modified:
        
        #count = count+ 2.12
        ax.axhline(y=k)
       
        if count%3 == 0:
            plt.text(y=k,s=k,x=count-5,color='blue',rotation=45,size ='large').set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])
        if count%3 == 1:
            plt.text(y=k,s=k,x=count+5,color='blue',rotation=45,size ='large').set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])
        if count%3 == 2:
            plt.text(y=k,s=k,x=count+16,color='blue',rotation=45, size ='large').set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])
        
        
        count = count +1 
    #plt.tight_layout()
    return plt.show()
    #return plt.show(), ml_results

In [879]:
# Function to create and set inital/placeholder values for size_delta, position, and sr_prox_value/line input features 
    # run once upon new scrape only
    
def feature_creation_inital():
    global inital_feature_run
    
    if inital_feature_run == False:
           
        # Set inital size delta l2 snapshot - use decimal point to force as float
        snapshot_asks_df['size_delta'] = 0.0
        snapshot_bids_df['size_delta'] = 0.0

        # Set inital position for l2 snapshot
        snapshot_asks_df['position'] = snapshot_asks_df.index +1
        snapshot_bids_df['position'] = snapshot_bids_df.index +1

        # Reverse index for sell/asks
        # snapshot_asks_df =snapshot_asks_df[::-1]
        # Reverse inital position indicator for bids/buy side
        rev = snapshot_bids_df['position'].values * -1
        snapshot_bids_df['position'] = rev

        # Set inital proximity value and inital value for nearest prox line
        snapshot_asks_df['sr_prox_value'] = 0
        snapshot_asks_df['sr_prox_line'] = 0
        snapshot_bids_df['sr_prox_value'] = 0
        snapshot_bids_df['sr_prox_line'] = 0
        
        # Create flag/boolean column for is_line feature - key prediction value for model
        snapshot_asks_df['is_line'] = 0
        snapshot_bids_df['is_line'] = 0
        
        # Set feature_creation_inital_run flag to True once run
        feature_creation_inital_run = True
        print("run")   
    elif inital_feature_run == True:
        # Required syntax for logical operator/ if-else structure, but does nothing as a result
        # i.e., a "pass"
        print("pass")
        pass

In [880]:
# Seperation of l2_updates into 4 15-min seperate l2 updates
     #l2update_df (1 hour of data) split into 4 15-min dataframes/csvs

def l2update_1hr_split(l2update_df):
    
    global l2update_15min_1
    global l2update_15min_2
    global l2update_15min_3
    global l2update_15min_3
    
    global l2update_time_df
    
    # 193848/4 = 48462
        # 48462
        # 96924
        # 145386
        # 193848
    l2update_15min_1=l2update_df[:48462]
    l2update_15min_2=l2update_df[48462:96924]
    l2update_15min_3=l2update_df[96924:145386]
    l2update_15min_4=l2update_df[145386:193848]
    l2update_15min_1.to_csv("raw_data/l2update_15min_1.csv",header=True,encoding='utf-8',index =False)
    l2update_15min_2.to_csv("raw_data/l2update_15min_2.csv",header=True,encoding='utf-8',index =False)
    l2update_15min_3.to_csv("raw_data/l2update_15min_3.csv",header=True,encoding='utf-8',index =False)
    l2update_15min_4.to_csv("raw_data/l2update_15min_4.csv",header=True,encoding='utf-8',index =False)
    
    # Add l2update_15min segement iso times to dataframe
    # Predicted vs actual for each 15 min section of L2 update
    # Create dataframe with start time of every l2update segement
    time.sleep(0.5)
    l2update_15min_1.reset_index(inplace=True,drop = True)
    l2update_15min_2.reset_index(inplace=True,drop = True)
    l2update_15min_3.reset_index(inplace=True,drop = True)
    l2update_15min_4.reset_index(inplace=True,drop = True)
    time.sleep(0.5)
    l2update_time_df=pd.DataFrame.from_dict(data={"l2update_15min_1":l2update_15min_1['time'][0],
                                "l2update_15min_2":l2update_15min_2['time'][0],
                                "l2update_15min_3":l2update_15min_3['time'][0],
                                "l2update_15min_4":l2update_15min_4['time'][0]},orient='index')

    # Change time label to iso for chart generation function
    l2update_time_df.columns = ["iso"]


In [881]:
#Function to join both sides of orderbook together into one dataframe
    # Buy/sell side labels need to be kept if combining 
    # Or use positive/negative values from position to determine buy/sell side labels from model output
    
def snapshot_join(snapshot_asks_df,snapshot_bids_df):
    global snapshot_both_df
    snapshot_both_df = pd.concat([snapshot_asks_df,snapshot_bids_df], axis=0, join='outer', join_axes=None, ignore_index=False,
              keys=None, levels=None, names=None, verify_integrity=False,
              copy=True)

In [882]:
# Function to update orderbook states and feature values 
    # Inputs (snapshot_both_df,l2update_15min)
        # Combined ask/sell snapshot required for this function + l2update state dataframe
        
    # Ideally applied to snapshot over defined interval (15m, in sync with chart update)
        # Removals: Orders with a 'size' of 0 are removed from the orderbook
        # Feature updates: size_delta, position, sr_prox_value, sr_prox_line, is_line
        
    # Must be run before model input in sequence
    
def apply_l2_update(snapshot_both_df,l2update_15min):
    
    global firstUpdate_both 
    
    snapshot_both_df = snapshot_both_df
    l2update_df = l2update_15min
    
    # Convert to array for loop value traversal        
    ml_results_modified_array = np.array(ml_results_modified)
    
    for i in range(len(snapshot_both_df)):

        # If size in l2update_df is 0, remove that row/price point from the orderbook state
        if l2update_df['size'].values[i] == 0:
            #print (snapshot_both_df['price'].values[i])
            snapshot_both_df.drop(snapshot_both_df.index[i])

        #First update to snapshot, set 'size_delta' to l2update values
            # Note: Keep here or move to end of feature_creation_inital depending on execution order
        elif l2update_df['size'].values[i] != 0 & firstUpdate_both == False:
            snapshot_both_df['size_delta'].values[i] = l2update_df['size'].values[i]
            firstUpdate_both = True
            
        #Subsequent size delta calc (if size not zero and firstUpdate_both == True):
            # Calcluate 'size_delta'
        elif l2update_df['size'].values[i] != 0:
            currentsize =snapshot_both_df['size'].values[i] 
            updatesize = l2update_df['size'].values[i]
            snapshot_both_df['size_delta'].values[i] = updatesize-currentsize

        # Update 'position' for orderbook state
        if snapshot_both_df['side'] is "buy":
            snapshot_both_df['position'] = snapshot_both_df.index + 1
            rev = snapshot_both_df['position'].values * -1
            snapshot_both_df['position'] = rev 
        elif snapshot_both_df['side'] is "sell":
            snapshot_asks_df['position'] = snapshot_asks_df.index + 1
        
        # Update 'is_line' state from ml_results_modified
        if snapshot_both_df['price'].values[i] in ml_results_modified: 
            snapshot_both_df['is_line'].values[i] = 1
    
    # Loop structure to update 'sr_prox_line','sr_prox_value' between orderbook states
    for i in range(len(snapshot_both_df['price'])):
        #if snapshot_both_df['price'].values[i] in ml_results_modified: 
            priceInt = int(snapshot_both_df['price'].values[i])
            diff =abs(ml_results_modified - priceInt)
            diff_array = np.array(diff)
            lowest_abs =min(diff)
            #print("=============================")
            #print("Price: ")
            #print(snapshot_both_df['price'].values[i])
            #print("Diff:")
            #print(diff)
            #print("lowest_abs value: ")
            #print(lowest_abs)
            minIndex= list(diff_array).index(lowest_abs)
            #print("Closest S/R line:")
            #print(ml_results_modified[minIndex])
            #print("=============================")

            #Set sr_prox_value to distance from closest S/R line
            snapshot_both_df['sr_prox_value'].values[i] = lowest_abs
            #Set sr_prox_line value
            snapshot_both_df['sr_prox_line'].values[i] = ml_results_modified[minIndex]    
    

In [883]:
###########################################################################

# Function execution order/testing 

# Load in data:
raw_csv_load()
snapshot_asks_df.head()

side    price      size
0  sell  9192.01  4.660822
1  sell  9194.95  1.086830
2  sell  9194.99  0.001500
3  sell  9195.00  2.061000
4  sell  9195.01  2.463947

In [884]:
# Perform first candlestick data request (start datetime from request_log):
chart_15m_request(request_log_df)
chart_15m_df.head()
#chart_15m_df.tail()

time     open     high      low    close    volume
299 2018-04-26 14:45:00  8760.50  8773.55  8755.00  8773.55  4.395267
298 2018-04-26 14:46:00  8773.55  8787.00  8773.54  8786.99  5.405845
297 2018-04-26 14:47:00  8787.00  8787.00  8786.99  8787.00  1.674196
296 2018-04-26 14:48:00  8787.00  8789.12  8786.99  8789.12  2.592617
295 2018-04-26 14:49:00  8789.12  8794.44  8789.12  8789.13  4.851487

In [885]:
# Plot the above data with autogenerated support and resistance levels for price 
    # Pass dataframe to autoSR() --> create chart + generate and plot auto S/R levels
    # Pass dataframe to autoSR() --> just generate S/R levels and return as array
autoSR(chart_15m_df), generate_chart(chart_15m_df)

(array([8773, 8792, 8794, 8803, 8813, 8815, 8820, 8822, 8833, 8834, 8846,
        8847, 8860, 8862, 8872, 8875, 8889, 8898, 8905, 8906]), None)

In [888]:
# Split l2update data from raw_data into 4 15-min segments
    # Pass l2update_df (1hr of data) into split function
    # output of this function: l2update_time_df
    
l2update_1hr_split(l2update_df)  

In [889]:
l2update_time_df.head()
#l2update_15min_1
#l2update_15min_2
#l2update_15min_3
#l2update_15min_3

iso
l2update_15min_1  2018-04-24T03:52:17.202Z
l2update_15min_2  2018-04-24T04:07:44.745Z
l2update_15min_3  2018-04-24T04:23:58.885Z
l2update_15min_4  2018-04-24T04:40:49.111Z

In [890]:
l2update_15min_1.head(2)

side    price    size                      time
0  buy  9170.51  0.0100  2018-04-24T03:52:17.202Z
1  buy  9191.16  0.0015  2018-04-24T03:52:17.210Z

In [891]:
l2update_15min_2.head(2)

side    price       size                      time
0   buy  9214.98  12.974719  2018-04-24T04:07:44.745Z
1  sell  9230.94   0.000000  2018-04-24T04:07:44.750Z

In [892]:
l2update_15min_3.head(2)

side    price    size                      time
0  buy  9132.01  0.0000  2018-04-24T04:23:58.885Z
1  buy  9066.07  0.0015  2018-04-24T04:23:58.887Z

In [893]:
l2update_15min_4.head(2)

side    price     size                      time
0  buy  9144.26  0.11244  2018-04-24T04:40:49.111Z
1  buy  9144.26  0.12043  2018-04-24T04:40:49.201Z

In [894]:
l2update_time_df['iso'][1]

'2018-04-24T04:07:44.745Z'

In [895]:
# Test chart data request function that accepts an iso date for request start
    # Currently not working; each request currently returns the same request timeframe-wise
    
def ch15m_req_time(request_start): #request_end as parameter?
    
    global chart_15m_df2
    request_start = request_start
    
    #request_end = request_end

    # Request 15 minutes of candlestick (open high low close) data from API
        # start time must be in ISO 8601 format for get_product_historic_rates()
    chart_15m = public_client.get_product_historic_rates('BTC-USD', start = request_start, granularity=60)
    
    #chart_15m = public_client.get_product_historic_rates('BTC-USD',\
                                                          #start = request_start,\
                                                          #end = request_end,\
                                                          #granularity=60)

    # Convert chart data response into dataframes
    chart_15m_df2 =pd.DataFrame.from_records(chart_15m,columns=["time","low", "high","open" ,"close", "volume"])

    # Reorder columns according to label order required by matplotlib finance package
    chart_15m_df2 = chart_15m_df2[["time","open","high","low","close","volume"]]

    # Reverse/sort timestamp order (without reversal chart labels/axis will be out of order)
    chart_15m_df2.sort_values(by='time',axis=0, inplace =True)

    #Convert/sort time to datetime object (for matplotlib chart format requirement)
    chart_15m_df2['time'] =pd.to_datetime(chart_15m_df2['time'],unit='s')

In [896]:
# Documentation for the above:
# https://github.com/danpaquin/gdax-python

    # Documentation for .get_product_historic_rates():
        # https://github.com/danpaquin/gdax-python/blob/master/gdax/public_client.py#L176
        
    # Gdax documentation on requesting historical rates: 
        # https://docs.gdax.com/#get-historic-rates

In [897]:
#ch15m_req_time(l2update_time_df['iso'][0])
ch15m_req_time(l2update_15min_1['time'][0])
chart_15m_1= chart_15m_df2
chart_15m_1.head()

time     open     high      low    close    volume
299 2018-04-26 14:45:00  8760.50  8773.55  8755.00  8773.55  4.395267
298 2018-04-26 14:46:00  8773.55  8787.00  8773.54  8786.99  5.405845
297 2018-04-26 14:47:00  8787.00  8787.00  8786.99  8787.00  1.674196
296 2018-04-26 14:48:00  8787.00  8789.12  8786.99  8789.12  2.592617
295 2018-04-26 14:49:00  8789.12  8794.44  8789.12  8789.13  4.851487

In [898]:
#ch15m_req_time(l2update_time_df['iso'][1])
ch15m_req_time(l2update_time_df['iso'][1])
chart_15m_2= chart_15m_df2
chart_15m_2.head()

time     open     high      low    close    volume
299 2018-04-26 14:45:00  8760.50  8773.55  8755.00  8773.55  4.395267
298 2018-04-26 14:46:00  8773.55  8787.00  8773.54  8786.99  5.405845
297 2018-04-26 14:47:00  8787.00  8787.00  8786.99  8787.00  1.674196
296 2018-04-26 14:48:00  8787.00  8789.12  8786.99  8789.12  2.592617
295 2018-04-26 14:49:00  8789.12  8794.44  8789.12  8789.13  4.851487

In [899]:
#ch15m_req_time(l2update_time_df['iso'][2])
ch15m_req_time(l2update_15min_3['time'][0])
chart_15m_3 =chart_15m_df2
chart_15m_3.head()

time     open     high      low    close    volume
299 2018-04-26 14:45:00  8760.50  8773.55  8755.00  8773.55  4.395267
298 2018-04-26 14:46:00  8773.55  8787.00  8773.54  8786.99  5.405845
297 2018-04-26 14:47:00  8787.00  8787.00  8786.99  8787.00  1.674196
296 2018-04-26 14:48:00  8787.00  8789.12  8786.99  8789.12  2.592617
295 2018-04-26 14:49:00  8789.12  8794.44  8789.12  8789.13  4.851487

In [900]:
#ch15m_req_time(l2update_time_df['iso'][3])
ch15m_req_time(l2update_15min_4['time'][0])
chart_15m_4 =chart_15m_df2
chart_15m_4.head()

time     open     high      low    close    volume
299 2018-04-26 14:45:00  8760.50  8773.55  8755.00  8773.55  4.395267
298 2018-04-26 14:46:00  8773.55  8787.00  8773.54  8786.99  5.405845
297 2018-04-26 14:47:00  8787.00  8787.00  8786.99  8787.00  1.674196
296 2018-04-26 14:48:00  8787.00  8789.12  8786.99  8789.12  2.592617
295 2018-04-26 14:49:00  8789.12  8794.44  8789.12  8789.13  4.851487

In [901]:
# Create inital features
feature_creation_inital()


run


In [902]:
snapshot_asks_df.head()

side    price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  sell  9192.01  4.660822         0.0         1              0             0   
1  sell  9194.95  1.086830         0.0         2              0             0   
2  sell  9194.99  0.001500         0.0         3              0             0   
3  sell  9195.00  2.061000         0.0         4              0             0   
4  sell  9195.01  2.463947         0.0         5              0             0   

   is_line  
0        0  
1        0  
2        0  
3        0  
4        0

In [903]:
snapshot_bids_df.head()

side    price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  buy  9192.00  7.480407         0.0        -1              0             0   
1  buy  9191.15  1.086510         0.0        -2              0             0   
2  buy  9191.11  0.201300         0.0        -3              0             0   
3  buy  9191.00  4.631669         0.0        -4              0             0   
4  buy  9190.26  2.333950         0.0        -5              0             0   

   is_line  
0        0  
1        0  
2        0  
3        0  
4        0

In [904]:
# Join asks/bids into one dataframe
snapshot_join(snapshot_asks_df,snapshot_bids_df)
snapshot_state0_df = snapshot_both_df
snapshot_both_df.count()

side             27819
price            27819
size             27819
size_delta       27819
position         27819
sr_prox_value    27819
sr_prox_line     27819
is_line          27819
dtype: int64

In [905]:
# Apply snapshot states + feature updates and create new dataframes for said states
    # i.e. creation of distinct dataframes for updated orderbook snapshots
apply_l2_update(snapshot_both_df,l2update_15min_1)
snapshot_state1_df = snapshot_both_df
snapshot_state1_df.head()

side    price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  sell  9192.01  4.660822      0.0100         1            286          8906   
1  sell  9194.95  1.086830      0.0015         2            288          8906   
2  sell  9194.99  0.001500      0.0000         3            288          8906   
3  sell  9195.00  2.061000      0.0000         4            289          8906   
4  sell  9195.01  2.463947      0.0015         5            289          8906   

   is_line  
0        0  
1        0  
2        0  
3        0  
4        0

In [906]:
apply_l2_update(snapshot_both_df,l2update_15min_2)
snapshot_state2_df = snapshot_both_df
snapshot_state2_df.head()

side    price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  sell  9192.01  4.660822   12.974719         1            286          8906   
1  sell  9194.95  1.086830    0.001500         2            288          8906   
2  sell  9194.99  0.001500    0.500000         3            288          8906   
3  sell  9195.00  2.061000    0.500000         4            289          8906   
4  sell  9195.01  2.463947   13.974719         5            289          8906   

   is_line  
0        0  
1        0  
2        0  
3        0  
4        0

In [907]:
apply_l2_update(snapshot_both_df,l2update_15min_3)
snapshot_state3_df = snapshot_both_df
snapshot_state3_df.head()

side    price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  sell  9192.01  4.660822   12.974719         1            286          8906   
1  sell  9194.95  1.086830    0.001500         2            288          8906   
2  sell  9194.99  0.001500    0.001500         3            288          8906   
3  sell  9195.00  2.061000    0.057000         4            289          8906   
4  sell  9195.01  2.463947    0.001500         5            289          8906   

   is_line  
0        0  
1        0  
2        0  
3        0  
4        0

In [908]:
apply_l2_update(snapshot_both_df,l2update_15min_4)
snapshot_state4_df = snapshot_both_df
snapshot_state4_df.head()

side    price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  sell  9192.01  4.660822    0.112440         1            286          8906   
1  sell  9194.95  1.086830    0.120430         2            288          8906   
2  sell  9194.99  0.001500    8.217109         3            288          8906   
3  sell  9195.00  2.061000    0.010000         4            289          8906   
4  sell  9195.01  2.463947    0.010000         5            289          8906   

   is_line  
0        0  
1        0  
2        0  
3        0  
4        0

In [909]:
# Verify position of 1 for first sell order
snapshot_both_df.loc[snapshot_both_df['position' ] == 1]

side    price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  sell  9192.01  4.660822     0.11244         1            286          8906   

   is_line  
0        0

In [910]:
# Verify position of -1 for first buy order
snapshot_both_df.loc[snapshot_both_df['position' ] ==- 1]

side   price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  buy  9192.0  7.480407         0.9        -1            286          8906   

   is_line  
0        0

In [911]:
#Check to make sure 'is_line' value was properly set
    # Display price levels that are equal to support and resistance values 
snapshot_both_df.loc[snapshot_both_df['is_line' ] == 1]

side   price       size  size_delta  position  sr_prox_value  \
953   buy  8906.0   0.272000    0.010000      -954              0   
954   buy  8905.0   8.525000    0.001500      -955              0   
997   buy  8889.0   0.225923    0.001000      -998              0   
1025  buy  8875.0   5.328573    0.000000     -1026              0   
1031  buy  8872.0   0.010000    0.001500     -1032              0   
1067  buy  8862.0   2.506951    6.380000     -1068              0   
1072  buy  8860.0   1.232377   12.533708     -1073              0   
1112  buy  8846.0   0.122600    0.900000     -1113              0   
1142  buy  8834.0   0.008000    0.010000     -1143              0   
1146  buy  8833.0   3.796200    0.001500     -1147              0   
1173  buy  8820.0   3.893999    0.001500     -1174              0   
1188  buy  8815.0  10.704703    0.342770     -1189              0   
1197  buy  8813.0   0.011400    1.800000     -1198              0   
1232  buy  8803.0   1.027360    0.001500     -1233              0   
1271  buy  8794.0   0.060000    0.001500     -1272              0   
1274  buy  8792.0   0.010000    0.500000     -1275              0   
1328  buy  8773.0   0.571206    0.001500     -1329              0   

      sr_prox_line  is_line  
953           8906        1  
954           8905        1  
997           8889        1  
1025          8875        1  
1031          8872        1  
1067          8862        1  
1072          8860        1  
1112          8846        1  
1142          8834        1  
1146          8833        1  
1173          8820        1  
1188          8815        1  
1197          8813        1  
1232          8803        1  
1271          8794        1  
1274          8792        1  
1328          8773        1

In [912]:
# Autogenerated Support and Resistance values 
ml_results_modified

array([8773, 8792, 8794, 8803, 8813, 8815, 8820, 8822, 8833, 8834, 8846,
       8847, 8860, 8862, 8872, 8875, 8889, 8898, 8905, 8906])

In [913]:
# Snaphot to use for first run/implementation of model:

In [914]:
snapshot_state1_df.head()

side    price      size  size_delta  position  sr_prox_value  sr_prox_line  \
0  sell  9192.01  4.660822    0.112440         1            286          8906   
1  sell  9194.95  1.086830    0.120430         2            288          8906   
2  sell  9194.99  0.001500    8.217109         3            288          8906   
3  sell  9195.00  2.061000    0.010000         4            289          8906   
4  sell  9195.01  2.463947    0.010000         5            289          8906   

   is_line  
0        0  
1        0  
2        0  
3        0  
4        0

In [915]:
# Categorical + Onehot encoding + fit_transform()

data = snapshot_state1_df.values
X = data[:,1:7]
y = data[:,7:8]
y = np.ravel(y)
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse=False)
encoded_y = encoded_y.reshape(len(encoded_y), 1)
onehot_y = onehot_encoder.fit_transform(encoded_y)

In [916]:
X

array([[9192.01, 4.6608220000000005, 0.11244, 1, 286, 8906],
       [9194.95, 1.08683, 0.12043, 2, 288, 8906],
       [9194.99, 0.0015, 8.21710931, 3, 288, 8906],
       ...,
       [0.03, 7258.221, 0.536045, -13739, 8773, 8773],
       [0.02, 25705.001, 0.9, -13740, 8773, 8773],
       [0.01, 152047.98092796, 0.0015, -13741, 8773, 8773]], dtype=object)

In [917]:
#from sklearn.preprocessing import RobustScaler, Normalizer

# Scale X input data/features

#X_scaler = StandardScaler().fit(X)
#X_scaled = X_scaler.transform(X)
#X_scaler
#X = X_scaled

In [918]:
# Test code for encoding of buy/sell column to one-hot
    # Not needed with position variables
        # Negative postion = buy side
        # Positive position = sell side

    #Can add as additional feature if needed
    
    
#encoded_X1 = label_encoder.fit_transform(X[:,0])
#encoded_X1 = encoded_x1.reshape(len(encoded_X1), 1)
#onehot_X1 = onehot_encoder.fit_transform(encoded_X1)
#onehot_X1

In [919]:
#pd.get_dummies(snapshot_state1_df['side'],prefix=None)

In [920]:
X

array([[9192.01, 4.6608220000000005, 0.11244, 1, 286, 8906],
       [9194.95, 1.08683, 0.12043, 2, 288, 8906],
       [9194.99, 0.0015, 8.21710931, 3, 288, 8906],
       ...,
       [0.03, 7258.221, 0.536045, -13739, 8773, 8773],
       [0.02, 25705.001, 0.9, -13740, 8773, 8773],
       [0.01, 152047.98092796, 0.0015, -13741, 8773, 8773]], dtype=object)

In [921]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [922]:
onehot_y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [923]:
# Shape information for input shaping/adjustment

print("Default shape:")
print(data.shape)
print("Shape without is_line column and buy/sell column :")
print(X.shape)
print("Shape of y:")
print(y.shape)
print("Shape of onehot_y:")
print(onehot_y.shape)

Default shape:
(27819, 8)
Shape without is_line column and buy/sell column :
(27819, 6)
Shape of y:
(27819,)
Shape of onehot_y:
(27819, 2)


In [924]:
# Reshaping of X input/features for LSTM model 
      # [samples, timesteps, features]
    
print(X.shape)
X_shaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))
print(X_shaped.shape)

(27819, 6)
(27819, 1, 6)


In [925]:
X_shaped.shape[0]

27819

In [926]:
X_shaped.shape[1]

1

In [927]:
X_shaped.shape[2]

6

In [928]:
onehot_y.shape

(27819, 2)

In [929]:
onehot_y.shape[1]

2

In [930]:
snapshot_state1_df.count()
# Calculation of batch size multipler for snapshot_state1_df:
#1 × 27819 = 27,819
#3 × 9273 = 27,819
#9 × 3091 = 27,819
#11 × 2529 = 27,819
#33 × 843 = 27,819
#99 × 281 = 27,819

side             27819
price            27819
size             27819
size_delta       27819
position         27819
sr_prox_value    27819
sr_prox_line     27819
is_line          27819
dtype: int64

In [961]:
# Command to reset state of each LSTM layer
model.reset_states()

In [962]:
### Model construction & Definition ###
from keras import optimizers
from keras.layers import Bidirectional


#rmsprop = optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)
#adam = optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#adadelta= optimizers.Adadelta(lr=1, rho=0.95, epsilon=1e-6, decay=0.0)
    #https://keras.io/optimizers/
#Nadam = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
sgd = optimizers.SGD(lr=0.03, decay=1e-6, momentum=1.5, nesterov=True)
    # required parameter for LSTM if stateful = True
    # Size of entire batch or batch sizes
#from keras.layers import LeakyReLU

data_dim = X_shaped.shape[2]
    # Features
    
timesteps = X_shaped.shape[1]
    # Timesteps
    
batch_size = 2529 #X_shaped.shape[0]
    # required parameter for LSTM if stateful = True
    # Size of entire batch or batch sizes
        # Calculation of batch size multipler for snapshot_state1_df:
        #1 × 27819 = 27,819
        #3 × 9273 = 27,819
        #9 × 3091 = 27,819
        #11 × 2529 = 27,819
        #33 × 843 = 27,819
        #99 × 281 = 27,819
    
model = Sequential()

model.add(Bidirectional(LSTM(128, return_sequences = True,
                            use_bias= False, bias_initializer = 'ones',
                            recurrent_dropout = 0.2, dropout = 0.2),
                        batch_input_shape=(batch_size , timesteps, data_dim )))    

model.add(Bidirectional(GRU(32, return_sequences=True,stateful=True,
                use_bias= True, bias_initializer = 'ones',  recurrent_dropout = 0.2, dropout = 0.2)))
model.add(Bidirectional(GRU(48, return_sequences=True,stateful=True,
                use_bias= True, bias_initializer = 'ones',  recurrent_dropout = 0.2, dropout = 0.2)))
model.add(Bidirectional(GRU(64, return_sequences=True,stateful=True,
                use_bias= False, bias_initializer = 'ones')))
model.add(Bidirectional(GRU(128, return_sequences=True,stateful=True,
                use_bias= False, bias_initializer = 'ones')))
model.add(Bidirectional(GRU(256, return_sequences=True,stateful=True,
                use_bias= True, bias_initializer = 'ones')))
model.add(Bidirectional(GRU(32, return_sequences=True,stateful=True,
                use_bias= False, bias_initializer = 'ones', recurrent_dropout = 0.2, dropout = 0.2)))

#model.add(LSTM(128, return_sequences=True, stateful=True,
               #batch_input_shape=(batch_size , timesteps, data_dim ),
               #use_bias= False, bias_initializer = 'ones',
               #recurrent_dropout = 0.2, dropout = 0.2))
    #input_shape(batch_size, timesteps,features)
    #input_shape(timesteps,features)

model.add(Bidirectional(LSTM(64, return_sequences=True,stateful=True,
               use_bias= False, bias_initializer = 'ones')))

model.add(Bidirectional(LSTM(32, return_sequences=False,stateful=True,
                use_bias= False, bias_initializer = 'ones')))




    #return_sequences= False for last LSTM to flatten output
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
# softmax, reduction in layer sizes, smaller more frequent bidirectional

#binary_crossentropy, adadelta
model.compile(loss = "binary_crossentropy", optimizer = 'sgd', metrics = ['accuracy'])


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_144 (Bidirecti (2529, 1, 256)            137216    
_________________________________________________________________
bidirectional_145 (Bidirecti (2529, 1, 64)             55488     
_________________________________________________________________
bidirectional_146 (Bidirecti (2529, 1, 96)             32544     
_________________________________________________________________
bidirectional_147 (Bidirecti (2529, 1, 128)            61440     
_________________________________________________________________
bidirectional_148 (Bidirecti (2529, 1, 256)            196608    
_________________________________________________________________
bidirectional_149 (Bidirecti (2529, 1, 512)            787968    
_________________________________________________________________
bidirectional_150 (Bidirecti (2529, 1, 64)             104448    
__________

In [963]:
#model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
#model.add(Dense(2, activation='sigmoid'))

#model.compile(loss = "categorical_crossentropy", optimizer = 'adagrad', metrics = ['accuracy'])
#model.compile(loss = "kullback_leibler_divergence", optimizer = 'adam', metrics = ['accuracy'])
#model.compile(loss = "logcosh", optimizer = 'adagrad', metrics = ['accuracy'])
#adam, adadelta

# tanh -1 to 1
# sigmoid 0 to 1

# Dropout configuration
    #https://arxiv.org/pdf/1312.4569.pdf
    #https://pdfs.semanticscholar.org/3061/db5aab0b3f6070ea0f19f8e76470e44aefa5.pdf

#model.add(LSTM(25, return_sequences=False,stateful=True,recurrent_dropout = 0.25, dropout = 0.25))
    #https://stackoverflow.com/questions/44924690/keras-the-difference-between-lstm-dropout-and-lstm-recurrent-dropout?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa

In [964]:
model.fit(
    X_shaped,
    onehot_y, 
    epochs=100,
    shuffle=False, #Set shuffle to false when stateful = True in model definition above
    verbose=2,
    batch_size=batch_size
    #,validation_data=(X_shaped, onehot_y)
)

Epoch 1/100
 - 15s - loss: 0.6830 - acc: 0.5407
Epoch 2/100
 - 0s - loss: 0.6548 - acc: 0.7664
Epoch 3/100
 - 0s - loss: 0.6313 - acc: 0.9724
Epoch 4/100
 - 0s - loss: 0.6097 - acc: 0.9988
Epoch 5/100
 - 0s - loss: 0.5891 - acc: 0.9994
Epoch 6/100
 - 0s - loss: 0.5694 - acc: 0.9994
Epoch 7/100
 - 0s - loss: 0.5507 - acc: 0.9994
Epoch 8/100
 - 0s - loss: 0.5325 - acc: 0.9994
Epoch 9/100
 - 0s - loss: 0.5147 - acc: 0.9994
Epoch 10/100
 - 0s - loss: 0.4981 - acc: 0.9994
Epoch 11/100
 - 0s - loss: 0.4820 - acc: 0.9994
Epoch 12/100
 - 0s - loss: 0.4663 - acc: 0.9994
Epoch 13/100
 - 0s - loss: 0.4513 - acc: 0.9994
Epoch 14/100
 - 0s - loss: 0.4367 - acc: 0.9994
Epoch 15/100
 - 0s - loss: 0.4227 - acc: 0.9994
Epoch 16/100
 - 0s - loss: 0.4093 - acc: 0.9994
Epoch 17/100
 - 0s - loss: 0.3962 - acc: 0.9994
Epoch 18/100
 - 0s - loss: 0.3839 - acc: 0.9994
Epoch 19/100
 - 0s - loss: 0.3717 - acc: 0.9994
Epoch 20/100
 - 0s - loss: 0.3598 - acc: 0.9994
Epoch 21/100
 - 0s - loss: 0.3486 - acc: 0.9994


In [965]:
model_loss, model_accuracy=model.evaluate(X_shaped, onehot_y,batch_size=batch_size)
print(f"Test model - Loss: {model_loss}, Accuracy: {model_accuracy}")

27819/27819 [==============================] - 5s 193us/step
Test model - Loss: 0.05229427767070857, Accuracy: 0.9993889060887423


In [966]:
model.predict(X_shaped,batch_size=batch_size)
bt = model.predict(X_shaped,batch_size=batch_size)
bt_df = pd.DataFrame(bt)
bt_df.index = snapshot_state1_df['price'].values
# Restore 'price' to results/index
bt_df.index.name = 'price'

In [967]:
model.predict(X_shaped,batch_size=batch_size)
bt = model.predict(X_shaped,batch_size=batch_size)
bt_df = pd.DataFrame(bt)
bt_df.index = snapshot_state1_df['price'].values
# Restore 'price' to results/index
bt_df.index.name = 'price'

In [968]:
bt_df.head(15)

0         1
price                      
9192.01  0.950813  0.050127
9194.95  0.952225  0.048651
9194.99  0.951148  0.049505
9195.00  0.950908  0.049036
9195.01  0.951470  0.049134
9196.00  0.950607  0.050002
9196.01  0.951060  0.049881
9196.02  0.951233  0.049643
9196.03  0.951244  0.049485
9196.04  0.951812  0.048598
9196.05  0.950875  0.049435
9196.55  0.951942  0.048511
9196.56  0.951011  0.049453
9196.79  0.950729  0.050078
9197.00  0.950034  0.049902

In [975]:
bt_df.tail(10)
# Model is picking up on the relationship between:
    # Round price points i.e. 0.10, 0.15, 100, 150 and S/R # Add back in buy/sell column distinction

0         1
price                    
0.10   0.951160  0.049713
0.09   0.951291  0.049520
0.08   0.950525  0.050479
0.07   0.952485  0.048349
0.06   0.950926  0.049408
0.05   0.950733  0.049968
0.04   0.951284  0.049010
0.03   0.951414  0.048716
0.02   0.951376  0.049643
0.01   0.951245  0.049597

In [976]:
# Autogenerated Support and Resistance values 
ml_results_modified

# Upper/lower bounds for dataframe query from min/max values
price_limit_lower=  (min(ml_results_modified) - 1000)
price_limit_upper = (max(ml_results_modified) + 1000)
query_df = bt_df[(bt_df.index >= price_limit_lower) & (bt_df.index <= price_limit_upper)]
print(query_df.count())
query_df.head(25)

0    6481
1    6481
dtype: int64


0         1
price                      
9192.01  0.950813  0.050127
9194.95  0.952225  0.048651
9194.99  0.951148  0.049505
9195.00  0.950908  0.049036
9195.01  0.951470  0.049134
9196.00  0.950607  0.050002
9196.01  0.951060  0.049881
9196.02  0.951233  0.049643
9196.03  0.951244  0.049485
9196.04  0.951812  0.048598
9196.05  0.950875  0.049435
9196.55  0.951942  0.048511
9196.56  0.951011  0.049453
9196.79  0.950729  0.050078
9197.00  0.950034  0.049902
9197.01  0.951098  0.049399
9198.00  0.951145  0.049136
9198.12  0.951355  0.049145
9198.88  0.950800  0.049926
9199.00  0.951439  0.049403
9200.00  0.950843  0.049810
9200.05  0.950951  0.050071
9201.00  0.950117  0.050492
9201.01  0.950612  0.049946
9202.00  0.950505  0.049913

In [977]:
query_df.tail(20)

0         1
price                      
7779.63  0.951482  0.048692
7779.23  0.950646  0.049826
7779.00  0.950987  0.049677
7777.98  0.950345  0.050358
7777.78  0.951205  0.049353
7777.77  0.951484  0.048743
7777.72  0.951383  0.049404
7777.70  0.951575  0.049512
7777.50  0.952111  0.048682
7777.03  0.950929  0.050261
7777.00  0.950805  0.049556
7776.47  0.950806  0.049454
7776.00  0.950756  0.050259
7775.84  0.951197  0.049481
7775.46  0.950663  0.049494
7775.11  0.951068  0.048979
7775.00  0.951458  0.049565
7774.74  0.950432  0.049983
7773.33  0.950703  0.050095
7773.00  0.950613  0.050694

In [978]:
# Autogenerated Support and Resistance values 
ml_results_modified

array([8773, 8792, 8794, 8803, 8813, 8815, 8820, 8822, 8833, 8834, 8846,
       8847, 8860, 8862, 8872, 8875, 8889, 8898, 8905, 8906])

In [973]:
bt_df.max()

0    0.952694
1    0.051929
dtype: float32

In [974]:
bt_df.idxmax()


0    9795.00
1     859.58
dtype: float64

In [960]:
# Model feature/x-input normalization/scaling
    # Defining relationships between variables
    # Taking into account variable scale/nature to scale/normalize effectively
# For model output results:
    # Add back in buy/sell column distinction
    # Limit N-range for selection?

In [ ]:
#bt_df.query('8740 <= price <= 8945')

In [ ]:
# Reverse cat + one-hot encoding
decoded = onehot_y.dot(onehot_encoder.active_features_).astype(int)
#print(decoded)
inverted = label_encoder.inverse_transform(decoded)
inverted

In [946]:
### Model load and save ###
# Save compiled model and model layer weights to disk

#model_json = model.to_json()
#with open("model\model_sgd_alt_extensivelayers.json", "w+") as json_file:
    #json_file.write(model_json)
#model.save_weights("model\model_sgd_alt_extensivelayers.h5")

In [ ]:
# Load model from disk and model layer weights

#json_file = open('model\model_sgd.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)

# load weights into new model
#loaded_model.load_weights("model\model_sgd.h5")
#print("Loaded model from disk")

In [ ]:

#loaded_model.compile(loss = "categorical_crossentropy", optimizer = 'rmsprop', metrics = ['accuracy'])
#loaded_model.predict(Xshapetest,batch_size=batch_size)
#loaded_model.summary()

In [ ]:
###########################################################################

In [ ]:
#### Preliminary code to address timezone issue for chart data scrape for API

print(request_log_df["iso"][1])
print(l2update_time_df["iso"][0])
# Actual -700 US/Mountain time conversion is:
    #2018-04-23T21:52:17.202-06:00
import pytz
from datetime import datetime
from dateutil.tz import *
test_time = l2update_time_df["iso"][0]
print(test_time)
test_time_dt = pd.to_datetime(test_time)
print(test_time_dt)
#test_time.astimezone(pytz.timezone('America/New_York'))
print(test_time_dt.tz_localize('utc'))
test_time_dt_localized= test_time_dt.tz_localize('utc')
print("Converted to EST:")
test_time_dt_localized.astimezone("US/Eastern")

In [ ]:
###########################################################################

In [ ]:
### l2_state_update() is Deprecated

# Function to update sr_prox_line and sr_prox_value between orderbook states
    # Note: l2_state_update applied first, then sr_update(); switch order if linear calls or switch execution block cell
def l2_state_update():
    
    global firstUpdate_bids 
    global firstUpdate_asks
    
    for i in range(len(snapshot_bids_df)):

        # If size in l2update_df is 0, remove that row/price point from the orderbook state
        if l2update_df['size'].values[i] == 0:
            #print (snapshot_bids_df['price'].values[i])
            snapshot_bids_df.drop(snapshot_bids_df.index[i])

        #First update to snapshot, set 'size_delta' to l2update values
            # Note: Keep here or move to end of feature_creation_inital depending on execution order
        elif l2update_df['size'].values[i] != 0 & firstUpdate_bids == False:
            snapshot_bids_df['size_delta'].values[i] = l2update_df['size'].values[i]
            firstUpdate_bids = True
        #Subsequent size delta calc
        elif l2update_df['size'].values[i] != 0:
            currentsize =snapshot_bids_df['size'].values[i] 
            updatesize = l2update_df['size'].values[i]
            snapshot_bids_df['size_delta'].values[i] = updatesize-currentsize

        # Update positional values for bids snapshot
        snapshot_bids_df['position'] = snapshot_bids_df.index + 1
        rev = snapshot_bids_df['position'].values * -1
        snapshot_bids_df['position'] = rev 
        
        # Update 'is_line' value
        if snapshot_bids_df['price'].values[i] in ml_results_modified: 
            snapshot_bids_df['is_line'].values[i] = 1

    for i in range(len(snapshot_asks_df)):

        # If size in l2update_df is 0, remove that row/price point from the orderbook state
        if l2update_df['size'].values[i] == 0:
            #print (snapshot_asks_df['price'].values[i])
            snapshot_asks_df.drop(snapshot_asks_df.index[i])

        #First update to snapshot, set 'size_delta' to l2update values
            # Note: Keep here or move to end of feature_creation_inital depending on execution order
        elif l2update_df['size'].values[i] != 0 & firstUpdate_asks == False:
            snapshot_asks_df['size_delta'].values[i] = l2update_df['size'].values[i]
            firstUpdate_asks = True
        #Subsequent size delta calc
        elif l2update_df['size'].values[i] != 0:
            currentsize =snapshot_asks_df['size'].values[i] 
            updatesize = l2update_df['size'].values[i]
            snapshot_asks_df['size_delta'].values[i] = updatesize-currentsize

        # Update positional values for_asks snapshot
        snapshot_asks_df['position'] = snapshot_asks_df.index + 1
        #rev = snapshot_asks_df['position'].values * -1
        #snapshot_asks_df['position'] = rev
        
        # Update 'is_line' value
        if snapshot_asks_df['price'].values[i] in ml_results_modified: 
            snapshot_asks_df['is_line'].values[i] = 1

### sr_update() is Deprecated

# Function to update sr_prox_line and sr_prox_value between orderbook states
    # Note: l2_state_update applied first, then sr_update(); switch order if linear calls or switch execution block cell
    # Must be run before model input in sequence
def sr_update():
    
    # Convert to array for loop value traversal        
    ml_results_modified_array = np.array(ml_results_modified)

    ### snapshot_asks sr_prox_line,sr_prox_value set ###
    for i in range(len(snapshot_asks_df['price'])):
        #if snapshot_asks_df['price'].values[i] in ml_results_modified: 
            priceInt = int(snapshot_asks_df['price'].values[i])
            diff =abs(ml_results_modified - priceInt)
            diff_array = np.array(diff)
            lowest_abs =min(diff)
            #print("=============================")
            #print("Price: ")
            #print(snapshot_asks_df['price'].values[i])
            #print("Diff:")
            #print(diff)
            #print("lowest_abs value: ")
            #print(lowest_abs)
            minIndex= list(diff_array).index(lowest_abs)
            #print("Closest S/R line:")
            #print(ml_results_modified[minIndex])
            #print("=============================")

            #Set sr_prox_value to distance from closest S/R line
            snapshot_asks_df['sr_prox_value'].values[i] = lowest_abs
            #Set sr_prox_line value
            snapshot_asks_df['sr_prox_line'].values[i] = ml_results_modified[minIndex]

    ### snapshot_bids sr_prox_line,sr_prox_value set ###
    for i in range(len(snapshot_bids_df['price'])):
        #if snapshot_bids_df['price'].values[i] in ml_results_modified: 
            priceInt = int(snapshot_bids_df['price'].values[i])
            diff =abs(ml_results_modified - priceInt)
            diff_array = np.array(diff)
            lowest_abs =min(diff)
            #print("=============================")
            #print("Price: ")
            #print(snapshot_bids_df['price'].values[i])
            #print("Diff:")
            #print(diff)
            #print("lowest_abs value: ")
            #print(lowest_abs)
            minIndex= list(diff_array).index(lowest_abs)
            #print("Closest S/R line:")
            #print(ml_results_modified[minIndex])
            #print("=============================")

            #Set sr_prox_value to distance from closest S/R line
            snapshot_bids_df['sr_prox_value'].values[i] = lowest_abs
            #Set sr_prox_line value
            snapshot_bids_df['sr_prox_line'].values[i] = ml_results_modified[minIndex]